In [2]:
import pandas as pd
import datetime as dt
import requests

In [4]:
members = pd.read_csv('Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62], dtype={'Empirically Attributed PCP Individual Provider NPI':'object', 'Empirically Attributed PCP Group Provider NPI':'object'})
admissions = pd.read_csv('Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12], dtype={'Service Provider NPI':'object', 'Provider Group NPI':'object'})

F:\Users\julie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
admissions_dedup = admissions.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = admissions_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
admissions.shape, admissions_dedup.shape

((6213, 23), (6149, 23))

In [6]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    last_index_event_encounter_id = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        ##set first event as an intiial event or a non-qualifying readmission as initial event
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            last_index_event_encounter_id = row['Encounter ID']
            readmission = False
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'index without readmission'
        ##check if qualifying readmission and change readmit status of most recently added initial event to true
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            admissions.loc[admissions['Encounter ID'] == last_index_event_encounter_id, 'type'] = 'index w readmission'
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'initial readmission'
        ##check for duplicate readmission
        else:
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'extra readmission'
            
    return patient_index_events, patient_readmission_events

In [7]:

total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in admissions_dedup['Patient ID'].unique():
    patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        admissions.loc[admissions['Encounter ID'] == encounter_id, 'type'] = 'index without readmission'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-7-45bfff96a76a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
<ipython-input-7-45bfff96a76a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


In [8]:
admissions['type'].value_counts()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
Name: type, dtype: int64

In [9]:
admissions[['type']] = admissions[['type']].fillna(value='duplicate encounter')

In [10]:
admissions['type'].value_counts()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64

In [11]:
admissions

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,type
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,index without readmission
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,index without readmission
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,index without readmission
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,index without readmission
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,index without readmission
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,index without readmission
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,index without readmission
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,index w readmission
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,initial readmission


In [30]:
admissions['Facility Provider ID'].value_counts().head(15)

DS1|NPI|1902803315    1204
DS1|NPI|1184722779    1196
DS1|NPI|1427055839     889
DS1|NPI|1689772592     679
DS1|NPI|1033298617     460
DS1|NPI|1710918545     436
DS1|NPI|1689608150     340
DS1|NPI|1710065933     109
DS1|NPI|0000000000     103
DS1|NPI|1992818256      87
DS1|NPI|1154428688      27
DS1|NPI|1902844988      26
DS1|NPI|1811080526      22
DS1|NPI|1174667158      21
DS1|NPI|1639172372      20
Name: Facility Provider ID, dtype: int64

In [29]:
adm_ra = admissions.loc[admissions['type'] == 'index w readmission']
adm_ra['Facility Provider ID'].value_counts().head(15)

DS1|NPI|1184722779    93
DS1|NPI|1902803315    83
DS1|NPI|1427055839    83
DS1|NPI|1689772592    60
DS1|NPI|1033298617    33
DS1|NPI|1710918545    32
DS1|NPI|1689608150    28
DS1|NPI|0000000000    11
DS1|NPI|1992818256     8
DS1|NPI|1710065933     6
DS1|NPI|1154428688     4
DS1|NPI|1952546616     3
DS1|NPI|1336328244     3
DS1|NPI|1841277704     3
DS1|NPI|1659359446     2
Name: Facility Provider ID, dtype: int64

In [33]:
hospitals = pd.DataFrame({"Facility Provider ID" : [93,1196,7.77], "Readmit Count" : [83,1204,6.89], "Total Count" : [83,889,9.33]})
hospitals

,1184722779,1902803315,1427055839
0,93.00,83.00,83.00
1,1196.00,1204.00,889.00
2,7.77,6.89,9.33


In [28]:
npi = pd.read_csv('Data Files/npi.csv')

In [27]:
hospital_wo_dups['facility provider']= ""
for index, elem in hospital_wo_dups.iterrows():
    res = npi.loc[npi['npi'] == elem['Facility Provider NPI'], 'entry'].reset_index().iloc[0].entry
    try:
        dict = eval(res)
        hospital_wo_dups.loc[index, 'facility provider'] = dict['basic']['name']
    except:
        hospital_wo_dups.loc[index, 'facility provider'] = "unavailable"

NameError: name 'hospital_wo_dups' is not defined